## 3.4 Pandas 数值运算方法

对于一元运算(像函数与三角函数),这些通用函数将在输出结果中**保留索引和列标签**; 而对于二元运算(如加法和乘法), Pandas在传递通用函数时会自动**对齐索引**进行计算. 这就意味着,保存数据内容和组合不同来源的数据---两处在NumPy数组中容易出错的地方,变成了Pandas的杀手锏.

### 3.4.1 通用函数: 保留索引
因为Pandas是建立在NumPy基础之上的, 所以NumPy的通用函数也同样适用于Pandas的Series和DataFrame.

In [32]:
import pandas as pd
import numpy as np

In [33]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0,10,4))
ser

0    6
1    3
2    7
3    4
dtype: int32

In [34]:
df = pd.DataFrame(rng.randint(0,10,(3,4)),columns=['a','b','c','d'])   # 列标签只有4个,必须和数组列数量一致
df

,a,b,c,d
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


### 3.4.2 通用函数: 索引对齐

当在两个Series或DataFrame对象进行二元计算时, Pandas会在计算过程中对齐两个对象的索引. 

Series索引对齐

In [35]:
rk = pd.Series({'湖北':1234,'湖南':3242,'广东':3233},name='人口')    # Series有个name属性, 可以作为DataFrame列名称吗
mj = pd.Series({'湖北':123,'湖南':321,'山东':21},name='面积')
rk /mj

山东          NaN
广东          NaN
湖北    10.032520
湖南    10.099688
dtype: float64

**结果数组的索引是两个数组的并集**. 也可以用另一种运算法则

In [36]:
rk.index | mj.index

Index(['山东', '广东', '湖北', '湖南'], dtype='object')

In [37]:
a = pd.Series([2,4,6],index=[0,1,2])
b = pd.Series([1,3,5],index=[1,2,3])
a + b
# 为什么不把缺失的当0处理呢?

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

In [38]:
a | b   # a 并 b 表示什么意思

0     True
1     True
2     True
3    False
dtype: bool

In [39]:
a & b  # a 交 b 表示什么意思

0    False
1    False
2    False
3    False
dtype: bool

In [40]:
a ^ b

0     True
1     True
2     True
3    False
dtype: bool

In [41]:
a * b

0     NaN
1     4.0
2    18.0
3     NaN
dtype: float64

In [42]:
a.add(b,fill_value=0)  # 缺失值当0处理

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

DataFrame索引对齐
在计算两个DataFrame时, 类似的索引对齐规则同样会出现在**共同列**中

In [43]:
A = pd.DataFrame(rng.randint(0,20,(2,2)),columns=list("AB"))
# 生成一个2*2的数组,数值都小于20. DataFrame列用list('AB')生成
B = pd.DataFrame(rng.randint(0,20,(3,3)),columns=list('ABC'))
print(A)
print('*'*20)
print(B)
print('*'*20)
print(A+B)

   A   B
0  1  11
1  5   1
********************
    A   B   C
0   0  11  11
1  16   9  15
2  14  14  18
********************
      A     B   C
0   1.0  22.0 NaN
1  21.0  10.0 NaN
2   NaN   NaN NaN


处理方法一: 用A中所有值得均值来填充缺失值. 计算A的均值需要用stack将二维数组压缩成一维数组

In [44]:
fill = A.stack().mean()
print(fill)
A.add(B,fill_value=fill)
# 比较下维度差,A数组缺的位置,都用均值填充

4.5


,A,B,C
0,1.0,22.0,15.5
1,21.0,10.0,19.5
2,18.5,18.5,22.5


Python运算符和Pandas方法的映射关系(下面必须空一行)

| Python运算符 | Pandas方法 |
| -- | -- |
| + | add() |
| - | sub(),subtract()|
| * | mul(),multiply()
| / | truediv(),div(),divide() |
| // | floordiv() |
| % | mod() |
| ** | pow() |

### 3.4.3 通用函数: DataFrame与Series的运算

一个二维**数组**减去自身的一行数据

In [45]:
rng = np.random.RandomState(42)
a = rng.randint(10,size=(3,4))
print(type(a))
a

<class 'numpy.ndarray'>


array([[6, 3, 7, 4],
       [6, 9, 2, 6],
       [7, 4, 3, 7]])

In [46]:
a - a[0]     # 0 表示第1行  

array([[ 0,  0,  0,  0],
       [ 0,  6, -5,  2],
       [ 1,  1, -4,  3]])

In [47]:
df = pd.DataFrame(a,columns=list('QRST'))
pd.DataFrame(a,columns=['q','r','s','t'])
pd.DataFrame(a,columns=('a','b','c','d'))
print(df)

   Q  R  S  T
0  6  3  7  4
1  6  9  2  6
2  7  4  3  7


一个**DataFrame**减去自身第1行

In [49]:
print(df-df.iloc[0])
print(df.subtract(df['R'],axis=0)   #此句报错, 可能是2.7 和3.+版本不兼容问题

SyntaxError: unexpected EOF while parsing (<ipython-input-49-0b4659832f8f>, line 2)

## 3.5 处理缺失值

三种方法: null, NaN 和NA.

识别缺失值的方法: 通过一个覆盖全局的**掩码**表示缺失值, 另一种方法用一个**标签纸**表示. 

### 3.5.2 Pandas的缺失值

Pandas最终选择用标签方法表示缺失值, 包括两种Python原有的缺失值: 浮点数据类型的NaN值, 以及Python的None对象. 后面我们将会发现,虽然这么做也有一些副作用, 但是在实际运用中够用.

**None: Python对象类型的缺失值**
    - None是一个Python单体对象, 经常在代码中表示缺失值. 由于None是一个Python对象, 所以不能作为任何NumPy/Pandas数组类型的缺失值, 只能用于'object'数据类型(即由Python对象构成的数组).   什么意思?

In [51]:
x1 = np.array([1,None,3,4])
x1

array([1, None, 3, 4], dtype=object)

使用Python对象构成的数组就意味着你对一个包含None的数组进行累计操作,如sum()和min(),通常会报错:   为什么?

In [52]:
x1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

**NaN: 数值类型的缺失值**
    - NaN(全程 Not a Number), 在任何系统中都兼容的特殊浮点数

In [55]:
x2 = np.array([1,np.nan,3,4])    # 要写成 np.nan
x2.dtype

dtype('float64')

NaN就像病毒一样会将它接触过的数据同化

In [56]:
1 + np.nan

nan

In [57]:
0 + np.nan

nan

In [58]:
x2.sum(),x2.max(),x2.min()

(nan, nan, nan)

**NumPy也提供了一些特殊的累计函数, 可以忽略缺失值的影响**

In [61]:
np.nansum(x2), np.nanmin(x2), np.nanmax(x2)
#  x2.nansum()   报错

(8.0, 1.0, 4.0)

**Pandas中NaN与None的差异**
    - Pandas把它们看成是可以等价交换的, 在适当的时候会将两者进行替换. 
    - Pandas会将没有标签值得数据类型自动转换成NA

### 3.5.3 处理缺失值

发现,剔除,替换数据中的缺失值的方法:
    - isnull(), 创建一个布尔类型的掩码标签缺失值
    - notnull(), 与isnull()相反的操作
    - dropna(), 返回一个剔除缺失值的数据
    - fillna(), 返回一个填充了缺失值的数据副本

**发现缺失值**
    - Pandas中有两种有效的方法可以发现缺失值: isnull()和notnull(),都返回布尔类型的掩码数据.

In [65]:
data = pd.Series([1,np.nan,'Hello',None])
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [67]:
data[data.notnull()]

0        1
2    Hello
dtype: object

**剔除缺失值**
    - dropna(),剔除缺失值
    - fillna(),填充缺失值

In [68]:
data.dropna()

0        1
2    Hello
dtype: object

而在DataFrame上使用时需要设置一些参数

In [69]:
df = pd.DataFrame([[1,np.nan,2],
                 [2,3,5],
                 [np.nan,4,6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


**没法从DataFrame中单独剔除一个值, 要么是剔除缺失值所在的整行,要么是整列. 根据实际需求.**
    - 默认情况下,dropna(),会剔除任何包含缺失值的整行数据

In [70]:
df.dropna()

,0,1,2
1,2.0,3.0,5


可以设置参数 axis=1 剔除任何包含缺失值的**整列**数据

In [71]:
df.dropna(axis=1)

,2
0,2
1,5
2,6


In [73]:
df[3] = np.nan    # 增加第4列,都为缺失值
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [75]:
df.dropna(axis='columns',how='all')    #  如果某一个列全为缺失值,则删除 ,how的参数还可以等于any, 默认是any

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


还可以通过参数thresh设置**非缺失值**的最小数量

**填充缺失值**

In [76]:
data = pd.Series([1,np.nan,2,None,3],index=list("abcde"))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

用一个单独的值来填充缺失值,比如0或者100

In [78]:
data.fillna(0)
data.fillna(100)

a      1.0
b    100.0
c      2.0
d    100.0
e      3.0
dtype: float64

可以用缺失值前面的有效值来从前往后填充

In [79]:
# 从前往后填充
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [80]:
# 从后往前填充
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

注意: 如果从前往后填充时, 需要填充的缺失值前面没有值, 那么它就仍然是缺失值.